In [ ]:
# Import Required libraries
import cv2
import os

# Replace with the path to your root directory containing the emotion folders
root_dir = '/content/drive/MyDrive/Untitled folder/Custom dataset/Game scne/Train'

# Replace with the desired output directory
output_dir = '/content/drive/MyDrive/Untitled folder/Custom dataset/Game scne/Test4'


# Function to preprocess an image
def preprocess_image(img_path):
  img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  img = cv2.resize(img, (150, 100))
  return img

# Iterate through each emotion folder
for emotion_folder in os.listdir(root_dir):
    emotion_path = os.path.join(root_dir, emotion_folder)

    # Create output folder for the emotion
    output_emotion_path = os.path.join(output_dir, emotion_folder)
    os.makedirs(output_emotion_path, exist_ok=True)

    # Iterate through images in the emotion folder
    for image_file in os.listdir(emotion_path):
        image_path = os.path.join(emotion_path, image_file)

        # Preprocess the image
        preprocessed_img = preprocess_image(image_path)

        # Save the preprocessed image in the output folder
        output_file = os.path.join(output_emotion_path, image_file)
        cv2.imwrite(output_file, preprocessed_img)

print("Image preprocessing complete!")

Image preprocessing complete!


In [ ]:
import os
import numpy as np
import cv2  # Make sure to import cv2 for image processing
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Replace with the path to your preprocessed data directory
data_dir = r'/content/drive/MyDrive/Untitled folder/Custom dataset/Game scne/Test4'  # Use raw string

# Parameters
img_height, img_width = 150,100
batch_size = 12
test_size = 0.20

# Load and preprocess data
data = []
labels = []

for emotion_folder in os.listdir(data_dir):
    emotion_path = os.path.join(data_dir, emotion_folder)
    for image_file in os.listdir(emotion_path):
        image_path = os.path.join(emotion_path, image_file)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (img_width, img_height))  # Ensure image is resized
        img = img.reshape((img_height, img_width, 1))
        data.append(img)
        labels.append(emotion_folder)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Encode labels
le = LabelEncoder()
labels = le.fit_transform(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=test_size, random_state=152)

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

print("Creating the CNN model...")
model2 = Sequential([
    Input(shape=(img_height, img_width, 1)),  # Use Input layer
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(le.classes_), activation='softmax')
])

# Compile the model
print("Compiling the model...")
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with a smaller batch size if necessary
print("Starting model training...")
from tensorflow.keras.callbacks import ReduceLROnPlateau

# adjusting learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-3)

#train model
history = model2.fit(
    train_datagen.flow(X_train, y_train, batch_size=16),  # Adjusted batch size
    epochs=15,
    validation_data=(X_test, y_test),
    callbacks=[reduce_lr]
)
# Evaluate the model
test_loss, test_acc = model2.evaluate(X_test, y_test)

print(f"Test accuracy: {test_acc}")

# Predict emotions from a new image
new_img = cv2.imread('/content/drive/MyDrive/Untitled folder/Custom dataset/Game scne/Test2/Disgust/Ankita_1.png', cv2.IMREAD_GRAYSCALE)
new_img = cv2.resize(new_img, (img_width, img_height))
new_img = np.expand_dims(new_img, axis=0)
new_img = new_img / 255.0

predicted_emotion = le.inverse_transform(np.argmax(model2.predict(new_img), axis=-1))[0]
print("Predicted emotion:", predicted_emotion)

Creating the CNN model...
Compiling the model...
Starting model training...
Epoch 1/15
10/10 [==============================] - 2s 74ms/step - loss: 163.0478 - accuracy: 0.1299 - val_loss: 2.2450 - val_accuracy: 0.4359 - lr: 0.0010
Epoch 2/15
10/10 [==============================] - 1s 55ms/step - loss: 1.8109 - accuracy: 0.2987 - val_loss: 1.7563 - val_accuracy: 0.2564 - lr: 0.0010
Epoch 3/15
10/10 [==============================] - 1s 57ms/step - loss: 1.7495 - accuracy: 0.3247 - val_loss: 1.5731 - val_accuracy: 0.3333 - lr: 0.0010
Epoch 4/15
10/10 [==============================] - 1s 53ms/step - loss: 1.7017 - accuracy: 0.2727 - val_loss: 1.5656 - val_accuracy: 0.3590 - lr: 0.0010
Epoch 5/15
10/10 [==============================] - 1s 52ms/step - loss: 1.6965 - accuracy: 0.3312 - val_loss: 1.5391 - val_accuracy: 0.3590 - lr: 0.0010
Epoch 6/15
10/10 [==============================] - 1s 56ms/step - loss: 1.6726 - accuracy: 0.3117 - val_loss: 1.5231 - val_accuracy: 0.3590 - lr: 0.001

In [ ]:
import os
import numpy as np
import cv2  # Make sure to import cv2 for image processing
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Replace with the path to your preprocessed data directory
data_dir = r'/content/drive/MyDrive/Untitled folder/Custom dataset/Game scne/Test4'

# Parameters
img_height, img_width = 150,100
batch_size = 12
test_size = 0.20

# Load and preprocess data
data = []
labels = []

for emotion_folder in os.listdir(data_dir):
    emotion_path = os.path.join(data_dir, emotion_folder)
    for image_file in os.listdir(emotion_path):
        image_path = os.path.join(emotion_path, image_file)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (img_width, img_height))  # Ensure image is resized
        img = img.reshape((img_height, img_width, 1))
        data.append(img)
        labels.append(emotion_folder)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Encode labels
le = LabelEncoder()
labels = le.fit_transform(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=test_size, random_state=48)

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

print("Creating the CNN model...")
model3 = Sequential([
    Input(shape=(img_height, img_width, 1)),  # Use Input layer
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(len(le.classes_), activation='softmax')
])

# Compile the model
print("Compiling the model...")
model3.compile(optimizer='RMSprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with a smaller batch size if necessary
print("Starting model training...")
from tensorflow.keras.callbacks import ReduceLROnPlateau

# adjusting learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-3)

#train model
history = model3.fit(
    train_datagen.flow(X_train, y_train, batch_size=16),  # Adjusted batch size
    epochs=20,
    validation_data=(X_test, y_test),
    callbacks=[reduce_lr]
)
# Evaluate the model
test_loss, test_acc = model3.evaluate(X_test, y_test)

print(f"Test accuracy: {test_acc}")

# Predict emotions from a new image
new_img = cv2.imread('/content/drive/MyDrive/Untitled folder/Custom dataset/Game scne/Test2/Disgust/Ankita_1.png', cv2.IMREAD_GRAYSCALE)
new_img = cv2.resize(new_img, (img_width, img_height))
new_img = np.expand_dims(new_img, axis=0)
new_img = new_img / 255.0

predicted_emotion = le.inverse_transform(np.argmax(model2.predict(new_img), axis=-1))[0]
print("Predicted emotion:", predicted_emotion)

Creating the CNN model...
Compiling the model...
Starting model training...
Epoch 1/20
10/10 [==============================] - 3s 181ms/step - loss: 348.8248 - accuracy: 0.1818 - val_loss: 1.9715 - val_accuracy: 0.2051 - lr: 0.0010
Epoch 2/20
10/10 [==============================] - 2s 152ms/step - loss: 3.4266 - accuracy: 0.2727 - val_loss: 12.8129 - val_accuracy: 0.2564 - lr: 0.0010
Epoch 3/20
10/10 [==============================] - 2s 162ms/step - loss: 2.8069 - accuracy: 0.4156 - val_loss: 1.2195 - val_accuracy: 0.4103 - lr: 0.0010
Epoch 4/20
10/10 [==============================] - 2s 153ms/step - loss: 2.1578 - accuracy: 0.3961 - val_loss: 1.3326 - val_accuracy: 0.5641 - lr: 0.0010
Epoch 5/20
10/10 [==============================] - 1s 147ms/step - loss: 2.1088 - accuracy: 0.3442 - val_loss: 1.3169 - val_accuracy: 0.4872 - lr: 0.0010
Epoch 6/20
10/10 [==============================] - 2s 150ms/step - loss: 1.5662 - accuracy: 0.3896 - val_loss: 1.1590 - val_accuracy: 0.6154 - lr

In [ ]:
# Save the model to a file
model3.save('Model3Custom_CK.h5', save_format='h5')
# Save the model to a file
model3.save('Model3Custom_CK.keras')  # Save as a single HDF5 file  # Save as a single HDF5 file